In [28]:
def combine(command, parameter, reuse = None):
    if(reuse == None):
        reuse = {}
    pointer = 0
    changed = False
    while(pointer < len(command)):
        line = ''
        count = 0
        no_space_start = pointer-1
        while(pointer < len(command)):
            if command[pointer] == '{':
                count += 1
            else:
                if(command[pointer] == ' ' or command[pointer] == '\t'):
                    line += command[pointer]
                else:
                    line = ''
                    no_space_start = pointer
                count = 0
            pointer += 1      
            if(count == 2):
                start = pointer
                break
            
        count = 0
        while(pointer < len(command)):
            if command[pointer] == '}':
                count += 1
            else:
                count = 0
            pointer += 1                
            if(count == 2):
                current = command[start:pointer-2].strip()
                if not current in reuse.keys() or reuse[current] == 0:
                    reuse[current] = 1
                else:
                    reuse[current] += 1
                    current = current + ' ' + str(reuse[current])
                if(current in parameter.keys()):
                    value = parameter[current]
                    if(len(value) > 0):
                        endn = value[-1] == '\n'
                        value = value.split('\n')
                        value = [i for i in value if i != '']
                        if len(value) > 1:
                            tv = ''
                            for v in value:
                                tv += line + v + '\n'
                        else:
                            tv = line + value[0] + ('\n' if endn else '')
                        changed = True
                    else:
                        tv = ''
                    command = command[:no_space_start+1] + tv + command[pointer:]
                    pointer = no_space_start + len(parameter[current])-1
                else:
                    reuse[current] = 0
                    command = command[:start] + ' ' + current + ' }}' + command[pointer:]
                break
    if(changed):
        command = combine(command, parameter, reuse)
    return command

code = '''{{ command }} {{ head }}:
    {{ body }}
'''

main = '''{{ body }}'''

foreach = combine(code,{
    'command': 'for',
    'head': '{{ pointer }} in {{ literable }}',
})

function_def = combine(code,{
    'command': 'def',
    'head': '{{ name }}({{ parameter }})'
})

function_call = '{{ name }}({{ parameter }})'


invoke = combine(function_call,{
    'parameter': ''
})

variable_def = '{{ name }} = {{ value }}\n'

write = combine(function_call,{
    'name':'print',
    'parameter':'{{ text }}'
})

_import = 'import {{ module }}\n'
from_import = 'from {{ module }} import {{ component }}\n'
from_import_all = combine(from_import,{
    'component': '*'
})

In [15]:
parameter = 'array'
pointer_1 = 'i'
pointer_2 = 'j'

function = combine(function_def,{
    'name': 'bar',
    'body': foreach,
    'parameter':parameter,
    'literable': parameter,
    'pointer': pointer_1,
    'body 2': foreach,
    'literable 2': pointer_1,
    'pointer 2': pointer_2,
    'body 3': write,
    'text': pointer_2
})

print(function)

def bar(array):
    for i in array:
        for j in i:
            print(j)





In [42]:

imports = combine(main,
{
    'body': 
            from_import_all * 5 +
            _import + '\n' +
            invoke + '\n' + 
            (function_call + '\n') * 2 +
            variable_def +
            function_call + '\n',
    'module'  : 'pygame.locals',
    'module 2': 'OpenGL.GL',
    'module 3': 'OpenGL.GLU',
    'module 4': 'OpenGL.GLUT',
    'module 5': 'geo_math',
    'module 6': 'pygame',
    'name': 'pygame.init',
    'name 2': 'pygame.display.set_caption',
    'parameter': '\'PyGame OpenGL\'',
    'name 3': 'pygame.display.set_icon',
    'parameter 2': 'pygame.image.load(\'icon.png\')',
    'name 4': 'screen',
    'value': '(800,600)',
    'name 5': 'pygame.display.set_mode',
    'parameter 3': 'screen, DOUBLEBUF|OPENGL',
},{})
print(imports)

from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
from geo_math import *
import pygame
pygame.init()
pygame.display.set_caption('PyGame OpenGL')
pygame.display.set_icon(pygame.image.load('icon.png'))
screen = (800,600)
pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)

